# Range Minimum Query

セグメント木の一種

DPとの応用  
http://topcoder.g.hatena.ne.jp/skyaozora/20171212/1513084670

In [108]:
from math import log2, ceil

INF = 10 ** 6

class RMQ:
    def __init__(self, x):
        self.n = 2 ** ceil(log2(len(x)))
        self.dat = [INF] * (2 * self.n - 1)
        
        for k, v in enumerate(x):
            self.update(k, v)
        
    def update(self, k, v):
        k += self.n - 1
        self.dat[k] = v
        while k > 0:
            k = (k - 1) // 2
            self.dat[k] = min(self.dat[k * 2 + 1], self.dat[k * 2 + 2])
            
    def query(self, a, b, k=0, l=0, r=None):
        # Retrun min [a, b]
        if r is None:
            r = self.n
        
        if r <= a or b <= l:
            return INF
        
        if a <= l and r <= b:
            return self.dat[k]
        
        vl = self.query(a, b, k * 2 + 1, l, (l + r) // 2)
        vr = self.query(a, b, k * 2 + 2, (l + r) // 2, r)
        
        return min(vl, vr)
            
    def __repr__(self):
        return ', '.join([str(v) for v in self.dat])

In [109]:
bit = [5, 3, 7, 9, 6, 4, 1]
rmq = RMQ(bit)
rmq

1, 3, 1, 3, 7, 4, 1, 5, 3, 7, 9, 6, 4, 1, 1000000

In [110]:
rmq.query(0, 10)

1

In [116]:
rmq.query(2, 5)

6

# Binary Index Tree

i番目までの数値の和を求める．

In [12]:
class BIT:
    # Binary Index Tree
    # Index start from 1.
    
    def __init__(self, bit):
        self.n = len(bit)
        self.bit = [0] * (self.n + 1)
        
        for i, v in enumerate(bit):
            self.update(i + 1, v)
        
    def update(self, i, v):
        while i <= self.n:
            self.bit[i] += v
            i += i & -i
            
    def sumup(self, i):
        s = 0
        while i > 0:
            s += self.bit[i]
            i -= i & -i
            
        return s
            
    def __repr__(self):
        return ', '.join([str(v) for v in self.bit])

In [13]:
bit = BIT([5, 3, 7, 9, 6, 4, 1, 2])
bit

0, 5, 8, 7, 24, 6, 10, 1, 37

In [14]:
bit.sumup(7)

35

In [16]:
bit.sumup(7) - bit.sumup(5)

5

# BIT alt

BITのデータ構造で，RMQと似たような操作も可能．  
ただし，これはMaximumを求めるBIT．

In [118]:
class BIT:
    def __init__(self, bit=None, n=None):
        if n is None:
            n = 10 ** 5 + 1

        self.n = n if bit is None else len(bit) + 1
        self.bit = [0] * self.n
        
        if bit is not None:
            for i, v in enumerate(bit):
                self.update(i + 1, v)
        
    def update(self, i, v):
        # 1-indexed
        while i <= self.n:
            if self.bit[i] < v:
                self.bit[i] = v
            self.bit[i] = max(self.bit[i], v)
            i += i & -i
            
    def query(self, i):
        # 1-indexed
        ret = 0
        while i > 0:
            if ret < self.bit[i]:
                ret = self.bit[i]
            i -= i & -i
            
        return ret
            
    def __repr__(self, max_len=100):
        return ', '.join([str(v) for v in self.bit[:max_len]])

In [119]:
x = [5, 3, 7, 9, 6, 4, 1, 2]
bit = BIT(bit=x)
bit

0, 5, 5, 7, 9, 6, 6, 1, 9

In [120]:
bit.query(3), bit.query(5)

(7, 9)

ちなみに，indexの演算はこの通り

In [121]:
for i in range(10):
    print(i, i & -i)

0 0
1 1
2 2
3 1
4 4
5 1
6 2
7 1
8 8
9 1


# Union Find Tree

疎集合データ構造  
N頂点の，単純とは限らない無向グラフの結合，連結判定．

https://atcoder.jp/contests/atc001/tasks/unionfind_a

In [1]:
N, Q = 8, 9
X = [[0, 1, 2], [0, 3, 2], [1, 1, 3], [1, 1, 4], [0, 2, 4], 
     [1, 4, 1], [0, 4, 2], [0, 0, 0], [1, 0, 0]]

In [10]:
# Parent
par = list(range(N))

# Rank of tree
rank = [0] * N

par, rank

([0, 1, 2, 3, 4, 5, 6, 7], [0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
def find(x):
    # Find root
    if par[x] == x:
        return x
    else:
        par[x] = find(par[x])
        return par[x]

In [12]:
def unite(x, y):
    # Unite tree
    x = find(x)
    y = find(y)
    
    if x == y:
        return
    
    if rank[x] < rank[y]:
        par[x] = y
    else:
        par[y] = x
        if rank[x] == rank[y]:
            rank[x] += 1

In [13]:
def same(x, y):
    # Check whether x & y belong to the same tree
    return find(x) == find(y)

In [14]:
for p, a, b in X:
    if p == 0:
        unite(a, b)
    else:
        print('Yes' if same(a, b) else 'No')

Yes
No
Yes
Yes


In [15]:
par, rank

([0, 1, 1, 1, 1, 5, 6, 7], [0, 1, 0, 0, 0, 0, 0, 0])

In [8]:
class UnionFind:
    def __init__(self, n):
        self.par = list(range(n))
        self.rank = [0] * n
        
    def find(self, x):
        if self.par[x] == x:
            return x
        else:
            self.par[x] = self.find(self.par[x])
            return self.par[x]
        
    def unite(self, x, y):
        x = self.find(x)
        y = self.find(y)
        
        if x == y:
            return
        
        if self.rank[x] == self.rank[y]:
            self.rank[x] += 1
        elif self.rank[x] < self.rank[y]:
            x, y = y, x
        
        self.par[y] = x
                
    def same(self, x, y):
        return self.find(x) == self.find(y)
    
    def __repr__(self):
        return ', '.join(str(x) for x in self.par)

In [9]:
N, Q = 8, 9
X = [[0, 1, 2], [0, 3, 2], [1, 1, 3], [1, 1, 4], [0, 2, 4], 
     [1, 4, 1], [0, 4, 2], [0, 0, 0], [1, 0, 0]]

In [10]:
t = UnionFind(N)
for p, a, b in X:
    if p == 0:
        t.unite(a, b)
    else:
        print('Yes' if t.same(a, b) else 'No')

Yes
No
Yes
Yes


In [11]:
t

0, 1, 1, 1, 1, 5, 6, 7

# 重み付き Union-Find

https://atcoder.jp/contests/arc090/tasks/arc090_b

In [2]:
class WeightedUnionFind:
    def __init__(self, n):
        self.par = list(range(n))
        self.rank = [0] * n
        self.weight = [0] * n
        
    def find(self, x):
        if self.par[x] == x:
            return x
        else:
            y = self.find(self.par[x])
            self.weight[x] += self.weight[self.par[x]]
            self.par[x] = y
            return y
        
    def unite(self, x, y, w):
        px = self.find(x)
        py = self.find(y)
        
        if px == py:
            return True
        
        if self.rank[px] == self.rank[py]:
            self.rank[px] += 1
        elif self.rank[px] < self.rank[py]:
            px, py = py, px
            x, y = y, x
            w *= -1

        self.par[py] = px
        self.weight[py] = self.weight[x] - self.weight[y] - w
                
    def same(self, x, y):
        return self.find(x) == self.find(y)
    
    def diff(self, x, y):
        return self.weight[x] - self.weight[y]

In [1]:
N, M = 3, 3
X = [[1, 2, 1], [2, 3, 1], [1, 3, 2]]

print('ans: Yes')

ans: Yes


In [5]:
t = WeightedUnionFind(N + 1)
for l, r, d in X:
    if t.same(l, r):
        if t.diff(l, r) != d:
            print('No')
            break
    else:
        t.unite(l, r, d)
else:
    print('Yes')

No
